### Alignment

An example of how to align samples in a workspace. This notebook assumes that you have initialized the workspace in the workspace/workspace_initialization example.

In [ ]:
import os

from protfarm.workspace import Workspace as ws
from protfarm.workspace import Database as db
from protfarm.workspace import Template
from protfarm.workspace import Alignment

In [ ]:
# The data path represents the location of all protein engineering sequencing experiments
DATA_PATH = os.path.join("..", "example_data")

# Each experiment is given its own name and subdirectory in the DATA PATH
# An experiment is a group of samples, all using the same variant region
EXPERIMENT_NAME = "demo"

TEMPLATE_SEQUENCE = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXGAGTGCCCAANNKNNKNNKNNKNNKNNKNNKGCACAGGCGCXXXXXXXXXXXXXXXXXXXX"
TEMPLATE_NAME = "7-mer NNK Insertion Position 56"

VARIANT_QUALITY_THRESHOLD = 30
MISMATCH_QUALITY_THRESHOLD = 20

# Currently, only Perfect_Match_Aligner is functional
ALIGNMENT_METHOD = "Perfect_Match_Aligner"

In [ ]:
# Set the data and experiment path
ws.set_data_path(DATA_PATH)
ws.set_experiment(EXPERIMENT_NAME)

In [ ]:
try:
    template = Template(TEMPLATE_SEQUENCE, name=TEMPLATE_NAME)
except Exception:
    template = db.get_template_by_name(TEMPLATE_NAME)

In [ ]:
sample_templates = {}

for sample in db.get_samples():
    sample_templates[sample.id] = template.id

alignment_parameters = {
    "mismatch_quality_threshold": MISMATCH_QUALITY_THRESHOLD,
    "variant_sequence_quality_threshold": VARIANT_QUALITY_THRESHOLD
}

try:
    alignment = Alignment(ALIGNMENT_METHOD,
                          parameters=alignment_parameters,
                          library_templates=sample_templates)
except ValueError:
    alignment = db.get_alignment_by_parameters(ALIGNMENT_METHOD, alignment_parameters, sample_templates)

In [ ]:
# Align all will align all unaligned samples

ws.align_all(print)

In [ ]:
# Get the results of this alignment and print the alignment statistics

alignment = db.get_alignments()[0]

for sample_id in alignment.statistics:
    
    sample_name = db.get_library_by_id(sample_id).name
    
    print("Alignment statistics for '%s'" % sample_name)
    
    for key, value in alignment.statistics[sample_id].items():
    
        print("%s: %.4f" % (key, value))
        
    print("")

In [ ]:
# This exports all alignment statistics to a file alignment_statistics.csv in the export folder
ws.export_alignment_statistics()